In [ ]:
from helpers import calculate_vector_field_eigenpairs
randomFieldV, jacobianV = calculate_vector_field_eigenpairs(10)

from helpers_diffusion import z_cov, z_calculate_random_field_eigenpairs
randomFieldZ = z_calculate_random_field_eigenpairs(10, z_cov)

In [ ]:
import numpy as np
xi_v = np.random.uniform(-np.sqrt(3), np.sqrt(3), randomFieldV.J)
xi_z = np.random.uniform(-np.sqrt(3), np.sqrt(3), randomFieldZ.J)

In [ ]:
# Diffusion û
import mshr
import numpy as np
import matplotlib.pyplot as plt
from helpers import DOMAIN, RHS_F
from helpers_diffusion import solve_diffusion_poisson_for_given_sample
from scipy.interpolate import griddata
import plotly.graph_objects as go
from matplotlib.ticker import MaxNLocator


u_sol = solve_diffusion_poisson_for_given_sample(10, RHS_F, randomFieldV, jacobianV, randomFieldZ, xi_v, xi_z)

mesh = mshr.generate_mesh(DOMAIN, 10)
x_coords = mesh.coordinates()[:, 0]
y_coords = mesh.coordinates()[:, 1]
grid_x, grid_y = np.mgrid[-1:1:500j, -1:1:500j]
u_sols = []

for i in range(len(x_coords)):
    u_sols.append(u_sol(x_coords[i], y_coords[i]))

grid_z = griddata((x_coords, y_coords), u_sols, (grid_x, grid_y), method='linear')

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
cp = ax.contourf(grid_x, grid_y, grid_z, levels=100, cmap='viridis')
cbar = plt.colorbar(cp)
cbar.ax.tick_params(labelsize=16)
cbar.locator = MaxNLocator(nbins=5)
cbar.update_ticks()
ax.set_xlim([-1.1, 1.1])
ax.set_ylim([-1.1, 1.1])
plt.title(r'$\hat{u}(\hat{x}, \omega)$ on $D_{ref}$', fontsize=24, pad=20)
plt.xlabel(r'$\hat{x}_1$', fontsize=24)
plt.ylabel(r'$\hat{x}_2$', fontsize=24)
ax.xaxis.set_major_locator(MaxNLocator(nbins=5))
ax.yaxis.set_major_locator(MaxNLocator(nbins=5))
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

In [ ]:
# Diffusion u
import mshr
import numpy as np
import matplotlib.pyplot as plt
from helpers import DOMAIN, RHS_F, inverse_mapping
from helpers_diffusion import solve_diffusion_poisson_for_given_sample
from scipy.interpolate import griddata
import plotly.graph_objects as go
import fenics as fe
from matplotlib.ticker import MaxNLocator


u_sol = solve_diffusion_poisson_for_given_sample(10, RHS_F, randomFieldV, jacobianV, randomFieldZ, xi_v, xi_z)

mesh = mshr.generate_mesh(DOMAIN, 10)
x_coords = mesh.coordinates()[:, 0]
y_coords = mesh.coordinates()[:, 1]
grid_x, grid_y = np.mgrid[-1.5:1.5:500j, -1.5:1.5:500j]
u_sols = []
x_coords_perturbed = []
y_coords_perturbed = []

for i in range(len(x_coords)):
    P_hat = fe.Point(x_coords[i], y_coords[i])
    x_coords_perturbed.append(randomFieldV(P_hat, xi_v)[0])
    y_coords_perturbed.append(randomFieldV(P_hat, xi_v)[1])
    u_sols.append(u_sol(P_hat))


grid_z = griddata((x_coords_perturbed, y_coords_perturbed), u_sols, (grid_x, grid_y), method='linear')

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
cp = ax.contourf(grid_x, grid_y, grid_z, levels=100, cmap='viridis')
cbar = plt.colorbar(cp)
cbar.ax.tick_params(labelsize=16)
cbar.locator = MaxNLocator(nbins=5)
cbar.update_ticks()
ax.set_xlim([np.min(x_coords_perturbed) - 0.1, np.max(x_coords_perturbed) + 0.1])
ax.set_ylim([np.min(y_coords_perturbed) - 0.1, np.max(y_coords_perturbed) + 0.1])
plt.title(r'$u(x, \omega)$ on $D(\omega)$', fontsize=24, pad=20)
plt.xlabel(r'$x_1$', fontsize=24)
plt.ylabel(r'$x_2$', fontsize=24)
ax.xaxis.set_major_locator(MaxNLocator(nbins=5))
ax.yaxis.set_major_locator(MaxNLocator(nbins=5))
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

In [ ]:
# plots for poisson mc-analysis
import fenics as fe
from helpers_diffusion import diffusion_analyse_two_resolutions_from_data_u_hat

diffusion_analyse_two_resolutions_from_data_u_hat(resolution_sparse = 10,
                                  resolution_fine = 14,
                                  P_hat = fe.Point(0.2, 0.2))


In [ ]:
# Sobol plot

from helpers import calculate_vector_field_eigenpairs
from helpers_diffusion import diffusion_plot_sobols, diffusion_sobol_calc_indices_from_data

mesh_res = 8
size_xi_v = 4
size_xi_z = 4
randomFieldV, jacobianV = calculate_vector_field_eigenpairs(mesh_res)

S_single, S_total, mc_sample_size = diffusion_sobol_calc_indices_from_data(fem_res=mesh_res, kl_res=mesh_res,
                                                                         size_xi_v=size_xi_v,
                                                                         size_xi_z=size_xi_z,
                                                                         randomFieldV=randomFieldV, jacobianV=jacobianV)
diffusion_plot_sobols(S_single, S_total, mc_sample_size, size_xi_v)

In [ ]:
from helpers_diffusion import *

randomFieldZ = z_calculate_random_field_eigenpairs(10, z_cov)

In [ ]:
xi_z = np.random.uniform(-np.sqrt(3), np.sqrt(3), randomFieldZ.J)
rectangular_domain = mshr.Rectangle(fe.Point(-1.5, -1.5), fe.Point(1.5, 1.5))

mesh = mshr.generate_mesh(rectangular_domain, 10)
x_coords = mesh.coordinates()[:, 0]
y_coords = mesh.coordinates()[:, 1]
grid_x, grid_y = np.mgrid[-1.5:1.5:500j, -1.5:1.5:500j]

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)

z_values = []
for i in range(len(x_coords)):
    z_values.append(randomFieldZ([x_coords[i], y_coords[i]], xi_z))
grid_z = griddata((x_coords, y_coords), z_values, (grid_x, grid_y), method='linear')
cp = ax.contourf(grid_x, grid_y, grid_z, levels=100, cmap='viridis')
cbar = plt.colorbar(cp)
cbar.ax.tick_params(labelsize=16)
cbar.locator = MaxNLocator(nbins=5)
cbar.update_ticks()
ax.set_xlabel(r'$x_1$', fontsize=20)
ax.set_ylabel(r'$x_2$', fontsize=20)
ax.set_xlim([-1.6, 1.6])
ax.set_ylim([-1.6, 1.6])
ax.xaxis.set_major_locator(MaxNLocator(nbins=5))
ax.yaxis.set_major_locator(MaxNLocator(nbins=5))
ax.tick_params(axis='both', which='major', labelsize=16)
ax.set_title(r'$Z(x, \omega_1)$', fontsize=24, pad=20)
plt.tight_layout()
plt.show()

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)

z_values = []
for i in range(len(x_coords)):
    z_values.append(np.exp(randomFieldZ([x_coords[i], y_coords[i]], xi_z)))
grid_z = griddata((x_coords, y_coords), z_values, (grid_x, grid_y), method='linear')
cp = ax.contourf(grid_x, grid_y, grid_z, levels=100, cmap='viridis')
cbar = plt.colorbar(cp)
cbar.ax.tick_params(labelsize=16)
cbar.locator = MaxNLocator(nbins=5)
cbar.update_ticks()
ax.set_xlabel(r'$x_1$', fontsize=20)
ax.set_ylabel(r'$x_2$', fontsize=20)
ax.set_xlim([-1.6, 1.6])
ax.set_ylim([-1.6, 1.6])
ax.xaxis.set_major_locator(MaxNLocator(nbins=5))
ax.yaxis.set_major_locator(MaxNLocator(nbins=5))
ax.tick_params(axis='both', which='major', labelsize=16)
ax.set_title(r'$A(x, \omega_1)$', fontsize=24, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Task: find out which Eigenvalues are used in sobol calculation

from helpers_diffusion import z_calculate_random_field_eigenpairs, z_cov
randomFieldZ = z_calculate_random_field_eigenpairs(8, z_cov)
print(f"randomFieldZ.eigenvalues[:4]: {randomFieldZ.eigenvalues[:4]}")

from helpers import calculate_vector_field_eigenpairs
randomFieldV = calculate_vector_field_eigenpairs(8)[0]
print(f"randomFieldV.eigenvalues[:4]: {randomFieldV.eigenvalues[:4]}")